# Inferencia de red ResNet para clasificación de signo a texto.

---

---


# Índice.

- [Configuración](#configuración)
  - [Configuración de la red](#configuración-de-la-red)
- [Elección del model a inferir](#elección-del-model-a-inferir)
  - [Carga del modelo](#carga-del-modelo)
- [Inferencia](#inferencia)
  - [Por webcam](#por-webcam)
  - [Por archivo](#desde-archivo)


## Configuración

---


In [1]:
from config.const import *
from config.dataset import get_dataset_path
from lib.video_dataset import VideoFrameDataset

c:\Users\Ivan\Documents\Projects\TFG\ubu-sign2text\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys

sys.path.append("../")

from common.config.torch_config import get_transform

### Configuración de la red


In [3]:
DATASETS = ["WLASL/videos", "actions/frames"]
MODELS_NAME = ["WLASL_8", "actions_small"]

index = 0

In [4]:
data_path, model_path = get_dataset_path(
    dataset=DATASETS[index], model_name=MODELS_NAME[index]
)
multiple_transform = get_transform(IMAGE_SIZE)

In [5]:
dataset = VideoFrameDataset(
    root_path=data_path,
    transform=multiple_transform,
    num_segments=NUM_SEGMENTS,
    frames_per_segment=FRAMES_PER_SEGMENT,
    image_size=IMAGE_SIZE,
)

classes = dataset.classes
classes

['before', 'book', 'chair', 'computer', 'drink']

## Elección del modelo a inferir

---


### Carga del modelo


In [6]:
from torch import load

In [7]:
model = load(model_path)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Ivan\\Documents\\Projects\\TFG\\ubu-sign2text\\models\\WLASL_8.pth'

### Carga del modelo onnx


In [ ]:
import onnx

In [ ]:
onnx_path = model_path.replace(".pth", ".onnx")
onnx_model = onnx.load(onnx_path)

In [ ]:
import onnxruntime as ort

providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]

In [ ]:
ort_session = ort.InferenceSession(
    onnx_path,
    providers=providers,
)

## Inferencia

---


In [ ]:
import sys

sys.path.append("../")

from common.inference import video_webcam_inference

### Con ONNX session


#### Test a random input on onnx model


In [ ]:
import numpy as np


def oxx_inference(video, session):
    outputs = session.run(
        None,
        {"input": video},
    )
    return classes[outputs[0][0].argmax(0)]

In [ ]:
target = oxx_inference(
    np.random.randn(
        1, FRAMES_PER_SEGMENT * NUM_SEGMENTS, 3, IMAGE_SIZE, IMAGE_SIZE
    ).astype(np.float32),
    ort_session,
)

print(target)

all


#### Ejemplo de video con onnx


In [ ]:
from utils.loader import split_dataset

In [ ]:
train_loader, test_loader, validation_loader = split_dataset(
    dataset, train_split=0.70, validation_split=0.1, batch_size=1
)

first_batch, (ground_classes, _) = next(iter(train_loader))
video = first_batch[0]
ground = classes[ground_classes[0]]

In [ ]:
normal_target = oxx_inference(first_batch.numpy(), ort_session)

In [ ]:
print(f"Target is {normal_target}. Ground truth is {ground}")

if normal_target == ground:
    print("Letsaaa gooo")

Target is drink. Ground truth is drink
Letsaaa gooo


### Probamos con modelo cuantizado


In [ ]:
quant_model_path = model_path.replace(".pth", "_quantized.onnx")
quant_onnx_model = onnx.load(quant_model_path)

In [ ]:
quant_ort_session = ort.InferenceSession(
    quant_model_path,
    providers=providers,
)

In [ ]:
quant_target = oxx_inference(first_batch.numpy(), quant_ort_session)

In [ ]:
print(
    f"Normal target is {normal_target}. Quant target is {quant_target}. Ground truth is {ground}"
)

Normal target is book. Quant target is book. Ground truth is book


### Por webcam con PyTorch


In [ ]:
video_webcam_inference(
    model,
    classes,
    "cuda",
    multiple_transform,
    fps_interval=NUM_SEGMENTS * FRAMES_PER_SEGMENT,
)